



```
# First import Necessary libraries
```




In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("customer_shopping_behavior.csv")


In [4]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [6]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [7]:
df.isnull().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [8]:
df["Review Rating"] = df.groupby("Category")['Review Rating'].transform(lambda x: x.fillna(x.median()))

In [9]:
df.columns = (
    df.columns
      .str.lower()
      .str.strip()
      .str.replace('[^a-z0-9]+', '_', regex=True)
      .str.replace('_+', '_', regex=True)
      .str.rstrip('_')
)


In [10]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_usd', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')



```
# Create a columns age_group
```



In [11]:
labels= ["Young_Adult", "Adult", "Middle_aged", "Senior"]
df["age_group"] = pd.qcut(df['age'], q=4, labels = labels)

In [12]:
df[['age', 'age_group']].head(10)

,age,age_group
0,55,Middle_aged
1,19,Young_Adult
2,50,Middle_aged
3,21,Young_Adult
4,45,Middle_aged
5,46,Middle_aged
6,63,Senior
7,27,Young_Adult
8,26,Young_Adult
9,57,Middle_aged




```
# Create column purchase_frequency_days
```



In [13]:
frequency_mapping = {
    'Fortnightly' : 14,
    'Weekly' : 7,
    'Montly' : 30,
    'Quarterly' : 90,
    'Bi-Weekly' : 14,
    'Annually' : 356,
    'Every 3 Months' :90
}

df['purchase_frequency_days']=  df['frequency_of_purchases'].map(frequency_mapping)

In [14]:
df[['purchase_frequency_days', 'frequency_of_purchases']].head()

,purchase_frequency_days,frequency_of_purchases
0,14.0,Fortnightly
1,14.0,Fortnightly
2,7.0,Weekly
3,7.0,Weekly
4,356.0,Annually


In [15]:
df[['discount_applied', "promo_code_used"]].head(10)

,discount_applied,promo_code_used
0,Yes,Yes
1,Yes,Yes
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,Yes,Yes
6,Yes,Yes
7,Yes,Yes
8,Yes,Yes
9,Yes,Yes


In [16]:
(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [17]:
df = df.drop('promo_code_used', axis=1)

In [18]:
!pip install psycopg2-binary sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/6e/b4/a5375cda5b54cb95ee9b836930fea30ae5a8f14aa97da7821722323d979b/psycopg2_binary-2.9.11-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/c7/23/907193c2f4d680aedbfbdf7bf24c13925e3c7c292e813326c1b84a0b878e/sqlalchemy-2.0.44-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for greenlet>=1 from https://files.pythonhosted.org/packages/d8/0f/30aef242fcab550b0b3520b8e3561156857c94288f0332a79928c31a52cf/greenlet-3.2.4-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 660.6 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.7 MB 2.8 MB/s eta 0:00:01
   ----- ---------------------------------- 0.

In [19]:
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
# Replace placeholders with your actual details

username = "postgres" # Default user
password = "2004" # the password you set during installation
host = 'localhost' # if running locally
port = '5432' # default PostgreSQL port
database = "Customer_Shopping_Behavior_Analysis" # the database you created in pdAdmin

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')



# Step 2: Load Dataframe into PostgreSQL
table_name = 'customer' # Choose any table name
df.to_sql(table_name, engine, if_exists='replace', index = False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}'.")

Data successfully loaded into table 'customer' in database 'Customer_Shopping_Behavior_Analysis'.
